In [21]:
import pandas as pd
import numpy as np
import itertools


N = 500
T = 50

u_i = np.random.uniform(0, 1, size=N)
x_i0 = np.random.binomial(1, p=u_i)

u = [u_i]
x = [x_i0]
y = [[None]*N]

for t in np.arange(1, T):
    u.append(u_i)
    x.append(np.random.binomial(1, p=1/(1 + np.exp(-2.*u_i))))
    #y.append(np.random.normal(6.*u_i + 3.*x[t]))
    y.append(np.random.normal(3.*x[t] + 3.*x[t-1]))

    
x = np.array(x)
u = np.array(u)
y = np.array(y)

df = pd.DataFrame([{"$i$": i, "$t$": t} for i, t in itertools.product(range(N), range(T))])
df['$X_{it}$'] = df[['$i$', '$t$']].apply(lambda row: x[row['$t$']][row['$i$']], axis=1)
df['$Y_{it}$'] = df[['$i$', '$t$']].apply(lambda row: y[row['$t$']][row['$i$']], axis=1)
df['$U_{it}$'] = df[['$i$', '$t$']].apply(lambda row: u[row['$t$']][row['$i$']], axis=1)

df = df.dropna()

In [22]:
df.corr()

,$i$,$t$,$X_{it}$,$Y_{it}$,$U_{it}$
$i$,1.000000e+00,-7.178252e-16,-0.008332,-0.013754,-4.972490e-02
$t$,-7.178252e-16,1.000000e+00,0.000406,0.008016,2.903778e-17
$X_{it}$,-8.332322e-03,4.062941e-04,1.000000,0.648224,2.433675e-01
$Y_{it}$,-1.375442e-02,8.015943e-03,0.648224,1.000000,3.043234e-01
$U_{it}$,-4.972490e-02,2.903778e-17,0.243367,0.304323,1.000000e+00


In [23]:
df[df['$t$'] < 6].groupby(['$X_{it}$', '$t$']).mean()

$i$  $Y_{it}$  $U_{it}$
$X_{it}$ $t$                                
0        1    246.560284  1.073544  0.369355
         2    235.175676  1.998636  0.431187
         3    258.978571  2.291140  0.410727
         4    247.859155  2.160430  0.436093
         5    249.139535  2.242140  0.388914
1        1    250.654596  4.709471  0.562788
         2    255.522727  5.242165  0.540637
         3    245.813889  5.059005  0.546161
         4    250.150838  5.197096  0.536857
         5    249.625337  5.145751  0.549730

In [24]:
from statsmodels.api import OLS


df['intercept'] = 1.

model = OLS(endog=df['$Y_{it}$'], exog=df[['$X_{it}$', 'intercept']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               $Y_{it}$   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                 1.775e+04
Date:                Tue, 06 Apr 2021   Prob (F-statistic):               0.00
Time:                        20:47:28   Log-Likelihood:                -47396.
No. Observations:               24500   AIC:                         9.480e+04
Df Residuals:                   24498   BIC:                         9.481e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$X_{it}$       3.1856      0.024    133.245      0.000       3.139       3.232
intercept      2.0303      0.020     99.874      0.000       1.990       2.070
==============================================================================
Omnibus:                     1266.573   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1214.342
Skew:                          -0.495   Prob(JB):                    2.04e-264
Kurtosis:                       2.543   Cond. No.                         3.57
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
df['intercept'] = 1.

model = OLS(endog=df['$Y_{it}$'], exog=df[['$X_{it}$', '$U_{it}$', 'intercept']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               $Y_{it}$   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     9743.
Date:                Tue, 06 Apr 2021   Prob (F-statistic):               0.00
Time:                        20:47:49   Log-Likelihood:                -46904.
No. Observations:               24500   AIC:                         9.381e+04
Df Residuals:                   24497   BIC:                         9.384e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
$X_{it}$       2.9992      0.024    124.146      0.000       2.952       3.047
$U_{it}$       1.1643      0.037     31.691      0.000       1.092       1.236
intercept      1.5732      0.025     63.963      0.000       1.525       1.621
==============================================================================
Omnibus:                      942.697   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              931.713
Skew:                          -0.439   Prob(JB):                    4.80e-203
Kurtosis:                       2.625   Cond. No.                         5.26
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""